# Import Packages

In [1]:
import sys
sys.path.append('..')
from mip import Model, xsum, INTEGER, CONTINUOUS, maximize, OptimizationStatus
import numpy as np
import pandas as pd
import utils



# Import Data



In [2]:
# Import Forecasts from File System
forecasts = pd.read_csv(utils.get_project_root() +
                        '/03_Forecasting_LightGBM/forecasts/forecasts.csv')
forecasts

,Week,Year,Site_ID,SKU,Prediction,Turnover
0,47,2020,101,1014557,30.889936,29.9
1,47,2020,101,1014824,25.798949,0.0
2,47,2020,101,1016525,57.913338,0.0
3,47,2020,107,1016525,55.792785,0.0
4,47,2020,590,1016525,55.746163,0.0
...,...,...,...,...,...,...
233639,44,2020,102,1281324700,0.806461,0.0
233640,44,2020,106,1281324700,0.806461,0.0
233641,44,2020,115,1281324700,0.806461,0.0
233642,44,2020,117,1281324700,0.806461,0.0


# Optimization (Example based on Article 1016525)

Problem:  
We would like to pass on a certain number of items from the central warehouse to the stores, but we don't have enough of them. Therefore, we need to reduce the number each store receives using an optimization.

Optimization is executed with the MIP solver (https://docs.python-mip.com/en/latest/index.html)

In [3]:
sku = 1016525

In [4]:
df = forecasts.loc[forecasts.SKU == sku].loc[forecasts.Week == 47]
df.sort_values(by='Site_ID')

,Week,Year,Site_ID,SKU,Prediction,Turnover
2,47,2020,101,1016525,57.913338,0.0
5405,47,2020,102,1016525,38.353527,0.0
5406,47,2020,103,1016525,36.140989,0.0
5407,47,2020,106,1016525,37.415935,0.0
3,47,2020,107,1016525,55.792785,0.0
5408,47,2020,115,1016525,36.140989,0.0
5409,47,2020,116,1016525,37.616108,0.0
26768,47,2020,117,1016525,9.328784,0.0
4,47,2020,590,1016525,55.746163,0.0


In [7]:
# Define Parameters
factor = 1.1 
n = 9        # numer of sites
supply = 300 # Avaialable stock at distribution center

# Demand of each site
d_1 = float(df.loc[df.Site_ID == 101]['Prediction'])
d_2 = float(df.loc[df.Site_ID == 102]['Prediction'])
d_3 = float(df.loc[df.Site_ID == 103]['Prediction'])
d_4 = float(df.loc[df.Site_ID == 106]['Prediction'])
d_5 = float(df.loc[df.Site_ID == 107]['Prediction'])
d_6 = float(df.loc[df.Site_ID == 115]['Prediction'])
d_7 = float(df.loc[df.Site_ID == 116]['Prediction'])
d_8 = float(df.loc[df.Site_ID == 117]['Prediction'])
d_9 = float(df.loc[df.Site_ID == 590]['Prediction'])
d = [d_1, d_2, d_3, d_4, d_5, d_6, d_7, d_8, d_9]

# Initiate the model
m = Model()

# Add Variables
x = [m.add_var(var_type=INTEGER, lb=1, ub=d[i]*factor) for i in range(n)]
p = m.add_var(var_type=CONTINUOUS, lb=0, ub=factor)

# Define Constraints
for i in range(n):
    m += d[i] * p <= x[i]
m += xsum(x[i] for i in range(n)) <= supply

# Define Target function
m.objective = maximize(p + xsum(x[i] for i in range(n))/supply)

# Optimize
status = m.optimize()

# Check if optimal solution was found
if not status == OptimizationStatus.OPTIMAL:
    raise AssertionError('Optimal solution was not found.')

# Print Results
print('p: ' + str(p.x))
for i in range(n):
    print(f'f_{i}: ', x[i].x)

p: 0.8082698568414332
f_0:  47.0
f_1:  31.0
f_2:  30.0
f_3:  31.0
f_4:  46.0
f_5:  30.0
f_6:  31.0
f_7:  8.0
f_8:  46.0


In [9]:
df.sort_values(by='Site_ID')

,Week,Year,Site_ID,SKU,Prediction,Turnover
2,47,2020,101,1016525,57.913338,0.0
5405,47,2020,102,1016525,38.353527,0.0
5406,47,2020,103,1016525,36.140989,0.0
5407,47,2020,106,1016525,37.415935,0.0
3,47,2020,107,1016525,55.792785,0.0
5408,47,2020,115,1016525,36.140989,0.0
5409,47,2020,116,1016525,37.616108,0.0
26768,47,2020,117,1016525,9.328784,0.0
4,47,2020,590,1016525,55.746163,0.0
